# Risks of Long-term Secondary Malignancies in Breast Cancer Patients Treated with Adjuvant Chemotherapy

[Benjamin Chan](http://careers.stackoverflow.com/benjaminchan)  

In [ ]:
message(sprintf("Run date: %s", Sys.time()))

Load packages.

In [ ]:
if (!require(openxlsx)) {install.packages("openxlsx", dependencies=TRUE, repos="https://cloud.r-project.org")}
library(openxlsx)
if (!require(data.table)) {install.packages("data.table", dependencies=TRUE, repos="https://cloud.r-project.org")}
library(data.table)

## Load data

Load the data from [GitHub](https://github.com/benjamin-chan/SecMalAfterBreastCaACT).

In [ ]:
repo <- "https://github.com/benjamin-chan/SecMalAfterBreastCaACT"
url <- paste0(repo, "/raw/26115dab1eb2b075d40f545514a03c3b30cde6b4/SecondaryMalignancies_Data_BG_3_4.xlsx")
f <- tempfile()
download.file(url, f, mode="wb")
file.info(f)[c("size", "mtime")]
sheets <- getSheetNames(f)
# sheets
D <- read.xlsx(f, sheet = sheets[1], startRow = 3, colNames = FALSE)
D <- data.table(D)
oldnames <- names(D)
newnames <- c("id",
              "authorYear",
              "trial",
              "quality",
              "arm",
              "nRandomized",
              "nITT",
              # "pctCompletingTx",
              "medianFU",
              "regimen",
              "anthracyclineType",
              "anthracyclineTotalDose",
              "anthracyclineDuration",
              "anthracyclineCourses",
              "cyclophosphamideDose",
              "cyclophosphamideDuration",
              "cyclophosphamideCourses",
              "taxaneType",
              "taxaneTotalDose",
              "taxaneDuration",
              "taxaneCourses",
              "fluoroucilTotalDose",
              "fluoroucilDuration",
              "fluoroucilCourses",
              "otherTxDetails",
              "malAML",
              "malMDS",
              "malAMLOrMDS",
              "malNonBreastSolid",
              "malNonBreastSolidType",
              "malOtherBloodCancers",
              "malSMRelatedDeaths",
              "malSecondPrimary",
              "NOTES")
setnames(D, oldnames, newnames[1:33])

## Tidy data

Create a `rowid` variable.
This will be handy later.

In [ ]:
D <- D[, rowid := .I]

Fix some minor data entry inconsistencies.

In [ ]:
D <- D[grep("Nitz", authorYear), authorYear := gsub("2014$", "2014)", authorYear)]
D <- D[grep("Wolmark", authorYear), authorYear := gsub(", 2001$", " (2001)", authorYear)]

Input missing data for Bergh (2000); row was split.

In [ ]:
col <- grep("^cyclophosphamide", names(D), invert = TRUE)
D[11, col] <- D[10, col, with = FALSE]
# D[11, ]
# D[10, ]

Fix trial column for Romond (2005).

In [ ]:
D <- D[grep("Romond", authorYear), trial := gsub("&N", "& N", trial)]

Make `medianFU` numeric.

In [ ]:
D <- D[grep("-", medianFU, invert = TRUE), medianFUNum := round(as.numeric(medianFU), digits = 2)]
x <- D[grep("-", medianFU), medianFU]
x <- as.numeric(unlist(strsplit(x, "-")))
x1 <- x[seq(1, length(x), 2)]
x2 <- x[seq(2, length(x), 2)]
xbar <- rowMeans(cbind(x1, x2))
D <- D[grep("-", medianFU), medianFUNum := xbar]
D <- D[,
       `:=` (medianFUChar = medianFU,
             medianFU = medianFUNum,
             medianFUNum = NULL)]
unique(D[as.character(medianFU) != medianFUChar, .(medianFU, medianFUChar)])
D <- D[, medianFUChar := NULL]

Recode mal* values of `NR` and `-` to `NA`.

In [ ]:
recode <- function(x) {
    missval <- c("-", "NR", " ")
    vec <- D[, get(x)]
    vec[vec %in% missval] <- NA
    vec
}
col <- grep("^mal", names(D), value = TRUE)
for (i in 1:length(col)) {
    D[, col[i]] <- recode(col[i])
}

Substitute non-numeric text in mal* values.

In [ ]:
substitute <- function(x) {
    vec <- D[, get(x)]
    vec <- gsub("[[:alpha:]]", "", vec)
    vec
}
col <- grep("^mal", names(D), value = TRUE)
for (i in 1:4) {
    # Only substitute a subset of mal* columns
    D[, col[i]] <- substitute(col[i])
}

Convert mal* values to numeric.

In [ ]:
for (i in 1:4) {
    # Only convert a subset of mal* columns
    D <- D[, col[i] := as.numeric(get(col[i]))]
}

The `malAMLOrMDS` column, as entered, captures number of AML or MDS malignancies if the study reported them grouped as opposed to separately.
If the study reported them as separately, then these counts are captures in the `malAML` and `malMDS` columns.
I.e., the `malAML`, `malMDS`, and `malAMLOrMDS` columns are mutually exclusive counts.
We want to create another column, `malAMLOrMDSTotal`, to be non-mutually exclusive from `malAML`, `malMDS`, and `malAMLOrMDS`.
If none of the `malAML`, `malMDS`, and `malAMLOrMDS` values are populated, then set `malAMLOrMDSTotal` to `NA`, also.
NOTE: Couldn't get this to work: [http://stackoverflow.com/a/16513949/1427069](http://stackoverflow.com/a/16513949/1427069).

In [ ]:
s <- sample(D[, rowid], 12)
malAMLOrMDSTotal <- rowSums(D[, .(malAML, malMDS, malAMLOrMDS)], na.rm=TRUE)
D <- D[, malAMLOrMDSTotal := malAMLOrMDSTotal]
D <- D[is.na(malAML) & is.na(malMDS) & is.na(malAMLOrMDS), malAMLOrMDSTotal := NA]
D[s, .(rowid, malAML, malMDS, malAMLOrMDS, malAMLOrMDSTotal)]

Remove text from `nITT` column.

In [ ]:
D <- D[authorYear == "Misset (1996)" & arm == 2, nITT := "137"]
D <- D[authorYear == "Fumoleu (2003)" & arm == 1, nITT := "210"]
D <- D[authorYear == "Fumoleu (2003)" & arm == 2, nITT := "197"]
D <- D[authorYear == "Fumoleu (2003)" & arm == 3, nITT := "195"]
D <- D[, nITT := as.integer(nITT)]

Clean up the `regimen` and `dose` columns.
Output to [regimens.md](regimens.md) for checking.

In [ ]:
D <- D[, regimen := gsub("\\s$", "", regimen)]
D <- D[, regimen := gsub("\\r\\n", " ", regimen)]
doseVar <- grep("dose", names(D), ignore.case = TRUE, value = TRUE)
for (i in 1:length(doseVar)) {
    D <- D[, doseVar[i] := gsub("\\r\\n", " ", get(doseVar[i]))]
}
write.table(D[, .N, c("regimen", doseVar), with = TRUE][order(regimen)],
            file = "regimens.md",
            sep = " | ", quote = FALSE,
            row.names = FALSE)
file.info("regimens.md")

Calculate cumulative dose: $\text{total dose} \times \text{number of courses}$.

In [ ]:
D <- D[, anthracyclineTotalDose := as.numeric(anthracyclineTotalDose)]
D <- D[authorYear == "Bergh (2000)" & regimen == "FEC\nTailored", anthracyclineTotalDose := 75]
D <- D[authorYear == "Henderson (2003)", anthracyclineTotalDose := 75]
D <- D[authorYear == "Colleoni (2009)" & regimen == "AC-CMF", anthracyclineTotalDose := 75]
D <- D[, cyclophosphamideDose := as.numeric(cyclophosphamideDose)]
D <- D[authorYear == "Colleoni (2009)" & regimen == "EC", cyclophosphamideDose := 600]
D <- D[authorYear == "Bergh (2000)" & regimen == "FEC\nTailored", cyclophosphamideDose := 900]
D <- D[authorYear == "Henderson (2003)", anthracyclineTotalDose := 75]
D <- D[, taxaneCourses := as.numeric(taxaneCourses)]
D <- D[authorYear == "Shulman (2014)" & regimen == "T" & arm == 3, taxaneCourses := 4]
D <- D[authorYear == "Shulman (2014)" & regimen == "T" & arm == 4, taxaneCourses := 6]
D <- D[, fluoroucilTotalDose := as.numeric(fluoroucilTotalDose)]
D <- D[authorYear == "Bergh (2000)" & regimen == "FEC\nTailored", fluoroucilTotalDose := 600]
D <- D[authorYear == "Joensuu (2012)" & regimen == "TX-CEX", fluoroucilTotalDose := NA]
D <- D[!is.na(anthracyclineTotalDose) & !is.na(anthracyclineTotalDose),
       anthracyclineCumulDose := anthracyclineTotalDose * anthracyclineCourses]
D <- D[!is.na(cyclophosphamideDose) & !is.na(cyclophosphamideCourses),
       cyclophosphamideCumulDose := cyclophosphamideDose * cyclophosphamideCourses]
D <- D[!is.na(as.numeric(taxaneTotalDose)) & !is.na(taxaneCourses),
       taxaneCumulDose := as.numeric(taxaneTotalDose) * taxaneCourses]
D <- D[!is.na(fluoroucilTotalDose) & !is.na(fluoroucilCourses),
       fluoroucilCumulDose := fluoroucilTotalDose * fluoroucilCourses]
# D <- D[is.na(anthracyclineCumulDose), anthracyclineCumulDose := 0]
# D <- D[is.na(cyclophosphamideCumulDose), cyclophosphamideCumulDose := 0]
# D <- D[is.na(taxaneCumulDose), taxaneCumulDose := 0]
# D <- D[is.na(fluoroucilCumulDose), fluoroucilCumulDose := 0]

Check.

In [ ]:
head(D[!is.na(anthracyclineCumulDose),
       .(authorYear, regimen, anthracyclineCumulDose, anthracyclineTotalDose, anthracyclineCourses)])
head(D[!is.na(cyclophosphamideCumulDose),
       .(authorYear, regimen, cyclophosphamideCumulDose, cyclophosphamideDose, cyclophosphamideCourses)])
head(D[!is.na(taxaneCumulDose),
       .(authorYear, regimen, taxaneCumulDose, taxaneTotalDose, taxaneCourses)])
head(D[!is.na(fluoroucilCumulDose),
       .(authorYear, regimen, fluoroucilCumulDose, fluoroucilTotalDose, fluoroucilCourses)])

### Manual tidying

After an early round of analysis, we realized there were some `nITT` data entry errors in the commit [26115da](https://github.com/benjamin-chan/SecMalAfterBreastCaACT/raw/26115dab1eb2b075d40f545514a03c3b30cde6b4/SecondaryMalignancies_Data_BG_3_4.xlsx) (Jun 1, 2016) spreadsheet.

In [ ]:
D <- D[authorYear == "Mamounas (2005)" & nITT == 1528, `:=` (nITTOld = nITT, nITT = 1529, isFixed = TRUE)]
D <- D[authorYear == "Fumoleu (2003)" & nITT == 210, `:=` (nITTOld = nITT, nITT = 212, isFixed = TRUE)]
D <- D[authorYear == "Fumoleu (2003)" & nITT == 197, `:=` (nITTOld = nITT, nITT = 209, isFixed = TRUE)]
D <- D[authorYear == "Fumoleu (2003)" & nITT == 195, `:=` (nITTOld = nITT, nITT = 200, isFixed = TRUE)]
D <- D[authorYear == "Fargeot (2004)" & nITT == 155, `:=` (nITTOld = nITT, nITT = 164, isFixed = TRUE)]
D <- D[authorYear == "Fargeot (2004)" & nITT == 163, `:=` (nITTOld = nITT, nITT = 174, isFixed = TRUE)]
D <- D[authorYear == "Kerbrat (2007)" & nITT == 235, `:=` (nITTOld = nITT, nITT = 241, isFixed = TRUE)]
D <- D[authorYear == "Kerbrat (2007)" & nITT == 236, `:=` (nITTOld = nITT, nITT = 241, isFixed = TRUE)]
D <- D[authorYear == "Fisher (1997)" & nITT == 763, `:=` (nITTOld = nITT, nITT = 767, isFixed = TRUE)]
D <- D[authorYear == "Fisher (1997)" & nITT == 771, `:=` (nITTOld = nITT, nITT = 772, isFixed = TRUE)]
D <- D[authorYear == "Bonneterre (2005)" & nITT == 271, `:=` (nITTOld = nITT, nITT = 289, isFixed = TRUE)]
D <- D[authorYear == "Bonneterre (2005)" & nITT == 266, `:=` (nITTOld = nITT, nITT = 276, isFixed = TRUE)]
D <- D[authorYear == "Roche (2006)" & nITT == 163, `:=` (nITTOld = nITT, nITT = 164, isFixed = TRUE)]
D <- D[authorYear == "Roche (2006)" & nITT == 168, `:=` (nITTOld = nITT, nITT = 169, isFixed = TRUE)]
D <- D[authorYear == "Shulman (2014)" & nITT == 1107, `:=` (nITTOld = nITT, nITT = 1142, isFixed = TRUE)]
D <- D[authorYear == "Shulman (2014)" & nITT ==  766, `:=` (nITTOld = nITT, nITT =  789, isFixed = TRUE)]
D <- D[authorYear == "Shulman (2014)" & nITT == 1119, `:=` (nITTOld = nITT, nITT = 1151, isFixed = TRUE)]
D <- D[authorYear == "Shulman (2014)" & nITT ==  782, `:=` (nITTOld = nITT, nITT =  789, isFixed = TRUE)]
D <- D[authorYear == "Swain (2013)" & nITT == 1617, `:=` (nITTOld = nITT, nITT = 1630, isFixed = TRUE)]
D <- D[authorYear == "Swain (2013)" & nITT == 1624, `:=` (nITTOld = nITT, nITT = 1634, isFixed = TRUE)]
D <- D[authorYear == "Swain (2013)" & nITT == 1618, `:=` (nITTOld = nITT, nITT = 1630, isFixed = TRUE)]
D[isFixed == TRUE, .(authorYear, regimen, nITTOld, nITT)]
D <- D[, `:=` (nITTOld = NULL, isFixed = NULL)]

We also made the decision to use the AML/MDS outcomes reported in Perez (2011) as the outcomes for Romond (2005) since they study the same cohort. Other edits to *AML or MDS* and *non-breast solid* malignancy outcome were needed, as well.

In [ ]:
D <- D[authorYear == "Martin (2010)" & regimen == "TAC",
      `:=` (malAMLOrMDSTotalOld = malAMLOrMDSTotal, malAMLOrMDSTotal = 0, isFixed = TRUE)]
D <- D[authorYear == "Martin (2010)" & regimen == "FAC",
      `:=` (malAMLOrMDSTotalOld = malAMLOrMDSTotal, malAMLOrMDSTotal = 0, isFixed = TRUE)]
D <- D[authorYear == "Romond (2005)" & regimen == "AC-T",
      `:=` (malAMLOrMDSTotalOld = malAMLOrMDSTotal, malAMLOrMDSTotal = 2, isFixed = TRUE)]
D <- D[authorYear == "Romond (2005)" & regimen == "ACT-T-Trast",
      `:=` (malAMLOrMDSTotalOld = malAMLOrMDSTotal, malAMLOrMDSTotal = 1, isFixed = TRUE)]
D <- D[authorYear == "Del Mastro (2015)",
      `:=` (malAMLOrMDSTotalOld = malAMLOrMDSTotal, malAMLOrMDSTotal = 0, isFixed = TRUE)]
D <- D[authorYear == "Del Mastro (2015)" & regimen == "EC-T" & nITT == 502,
      `:=` (malAMLOrMDSTotalOld = malAMLOrMDSTotal, malAMLOrMDSTotal = 2, isFixed = TRUE)]

D <- D[authorYear == "Bernard-Marty (2003)" & regimen == "EC" & nITT == 267,
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 4, isFixed = TRUE)]
D <- D[authorYear == "Bonneterre (2005)" & regimen == "FEC 50",
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 11, isFixed = TRUE)]
D <- D[authorYear == "Bonneterre (2005)" & regimen == "FEC 100",
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 15, isFixed = TRUE)]
D <- D[authorYear == "Citron (2003)" & regimen == "A-T-C" & nITT == 493,
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 11, isFixed = TRUE)]
D <- D[authorYear == "Citron (2003)" & regimen == "AC-T" & nITT == 495,
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 11, isFixed = TRUE)]
D <- D[authorYear == "Eiermann (2011)" & regimen == "AC-T",
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 34, isFixed = TRUE)]
D <- D[authorYear == "Eiermann (2011)" & regimen == "TAC",
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 31, isFixed = TRUE)]
D <- D[authorYear == "Romond (2005)" & regimen == "AC-T",
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 13, isFixed = TRUE)]
D <- D[authorYear == "Romond (2005)" & regimen == "ACT-T-Trast",
      `:=` (malNonBreastSolidOld = malNonBreastSolid, malNonBreastSolid = 4, isFixed = TRUE)]

D[isFixed == TRUE,
  .(authorYear, regimen, nITT, malAMLOrMDSTotalOld, malAMLOrMDSTotal, malNonBreastSolidOld, malNonBreastSolid)]
D <- D[, `:=` (malAMLOrMDSTotalOld = NULL, malNonBreastSolidOld = NULL, isFixed = NULL)]

Deduplicate.

In [ ]:
n0 <- nrow(D)
setkey(D, authorYear, trial, arm)
D <- unique(D)
message(sprintf("Removed %d duplicate row(s) (%.03g%%)", n0 - nrow(D), (n0 - nrow(D)) / n0 * 100))

## Summarize

Define some functions.

In [ ]:
scale <- 1e4
calcPct <- function (x, n) {
  prec <- "%.3g"
  sprintf(paste0(prec, "%%"),
          median(x / n, na.rm = TRUE) * 100)
}
calcRate <- function (x, n, y) {
  prec <- "%.3g"
  py <- scale
  sprintf(paste(prec, "per %s p-y"),
          median(x / (n * (y / 12)), na.rm=TRUE) * py,
          py)
}

Summarize the regimens.
Output to [summaryRegimens.md](summaryRegimens.md) for checking.

In [ ]:
D <- D[,
       `:=` (isAnthra = !is.na(anthracyclineTotalDose),
             isCyclo = !is.na(cyclophosphamideDose),
             isTaxane = !is.na(taxaneTotalDose),
             isFluoro = !is.na(fluoroucilTotalDose))]
D1 <- melt(D,
           id.vars=c("id", "authorYear", "arm",
                     "isAnthra", "anthracyclineCumulDose",
                     "isCyclo", "cyclophosphamideCumulDose",
                     "isTaxane", "taxaneCumulDose",
                     "isFluoro", "fluoroucilCumulDose",
                     "nITT", "medianFU"),
           measure.vars=c("malAML", "malMDS", "malAMLOrMDSTotal", "malNonBreastSolid"),
           value.name="malN",
           variable.name="malType")

D1 <- D1[, malType := gsub("^mal", "", malType)]
D1 <- D1[, malType := factor(malType,
                             levels=c("AML", "MDS", "AMLOrMDSTotal", "NonBreastSolid"),
                             labels=c("AML", "MDS", "AML or MDS", "Non-Breast Solid"))]
D1 <- D1[, py := nITT * (medianFU / 12)]
D1 <- D1[, rate := malN / py * scale]
summaryRegimens <- D1[,
                      .(totalN = sum(nITT, na.rm = TRUE),
                        totalPersonYears = round(sum(py, na.rm = TRUE)),
                        totalMalignancies = sum(malN, na.rm = TRUE),
                        medianPct = calcPct(malN, nITT),
                        medianRate = calcRate(malN, nITT, medianFU)),
                      .(isAnthra,
                        isCyclo,
                        isTaxane,
                        isFluoro,
                        malType)]
summaryRegimens <- summaryRegimens[order(-isAnthra, -isCyclo, -isTaxane, -isFluoro, malType)]
write.table(summaryRegimens,
            file = "summaryRegimens.md",
            sep = " | ", quote = FALSE,
            row.names = FALSE)
file.info("summaryRegimens.md")

## Meta-regression

Estimate meta-regression models for log transformed incidence rate.
Model is

$$\frac{y_i}{\text{py}_i} = \beta x_i + \sigma_\text{study}$$

Or

$$\frac{y_i}{\text{py}_i} = \beta I_{\text{high dose}, i} + \sigma_\text{study}$$

Models were estimated using the `rma.mv()` function from the metafor` package for R.

In [ ]:
library(metafor)
citation("metafor")

Define meta-regression functions.

In [ ]:
pvalToChar <- function (p) {
  if (p < 0.001) {
    pvalue <- "p < 0.001"
  } else {
    pvalue <- sprintf("p = %.03f", p)
  }
  pvalue
}
metareg <- function (D) {
  require(metafor)
  D <- D[!(is.na(x) | is.na(rate) | is.na(nITT) | is.na(malType))]
  xData <- unique(D[, .(drug, x, xHighDose, malType)])
  D <- escalc("IRLN", xi=malN, ti=py, data=D)
  randomEffect <- list(~ 1 | id)
  MLin <- rma.mv(yi ~ x, vi, random=randomEffect, data=D)
  MBin <- rma.mv(yi ~ xHighDose, vi, random=randomEffect, data=D)
  pvalueLin <- MLin$pval[which(row.names(MLin$b) == "x")]
  pvalueBin <- MBin$pval[which(row.names(MBin$b) == "xHighDoseTRUE")]
  predLin <- predict(MLin, xData[, x], transf = exp)[["pred"]] * scale
  confLowerLin <- predict(MLin, xData[, x], transf = exp)[["cr.lb"]] * scale
  confUpperLin <- predict(MLin, xData[, x], transf = exp)[["cr.ub"]] * scale
  predBin <- predict(MBin, xData[, xHighDose], transf = exp)[["pred"]] * scale
  confLowerBin <- predict(MBin, xData[, xHighDose], transf = exp)[["cr.lb"]] * scale
  confUpperBin <- predict(MBin, xData[, xHighDose], transf = exp)[["cr.ub"]] * scale
  pred <- data.table(xData, predLin, confLowerLin, confUpperLin, predBin, confLowerBin, confUpperBin, scale)
  setorder(pred, x)
  list(rmaLin = MLin,
       rmaBin = MBin,
       pvalueLin = pvalToChar(pvalueLin),
       pvalueBin = pvalToChar(pvalueBin),
       pred = pred)
}
plotreg <- function (M, D, title, xlab, xbreaks, xscale) {
  require(ggplot2)
  require(RColorBrewer)
  require(tools)
  pvalues <- c(M$pvalueLin, M$pvalueBin)
  x <- seq(min(M$rmaLin$X[, "x"]), max(M$rmaLin$X[, "x"]), length.out=100)
  yhat1 <- unique(data.table(malType = mal,
                             x,
                             yhat = predict(M$rmaLin, x, transf = exp)[["pred"]] * scale))
  xHighDose <- as.logical(M$rmaBin$X[, "xHighDoseTRUE"])
  yhat2 <- unique(data.table(malType = mal,
                             xHighDose,
                             yhat = predict(M$rmaBin, xHighDose, transf = exp)[["pred"]] * scale))
  D <- D[!(is.na(x) | is.na(rate) | is.na(nITT) | is.na(malType))]
  D <- D[, malType := droplevels(malType)]
  steps <- merge(D[, .(min = min(x), max = max(x)), .(malType, xHighDose)],
                 yhat2,
                 by=c("malType", "xHighDose"))
  steps <- melt(steps, id.vars=c("malType", "xHighDose", "yhat"), measure.vars=c("min", "max"), value.name="x")
  annoLin <- data.frame(x=Inf, y=1.2, label=pvalues[c(1)], malType=levels(D[, malType]))
  annoBin <- data.frame(x=Inf, y=1.4, label=pvalues[c(2)], malType=levels(D[, malType]))
  pal <- brewer.pal(4, name="RdBu")
  G <- ggplot(D, aes(x=x * xscale, y=rate + 1/2, size=nITT / min(nITT, na.rm=TRUE)))
  G <- G + geom_point(alpha=1/3, position="jitter")
  G <- G + geom_line(data=yhat1, aes(x=x * xscale, y=yhat), inherit.aes=FALSE, color=pal[4])
  G <- G + geom_step(data=steps, aes(x=x * xscale, y=yhat), inherit.aes=FALSE, color=pal[1])
  G <- G + geom_text(data=annoLin,
                     aes(x, y, label=label), inherit.aes=FALSE, hjust=1, color=pal[4])
  G <- G + geom_text(data=annoBin,
                     aes(x, y, label=label), inherit.aes=FALSE, hjust=1, color=pal[1])
  G <- G + scale_x_log10(xlab, breaks=xbreaks)
  G <- G + scale_y_log10(sprintf("Rate per %s person-years", format(scale, big.mark=",")))
  G <- G + labs(title=title)
  G <- G + theme_bw()
  G <- G + theme(legend.position="none")
  filename <- gsub("\\s+",
                   "",
                   toTitleCase(paste(gsub("[[:punct:]]", "", title),
                                     gsub("cumulative dose", "", xlab),
                                     sep="_")))
  ggsave(filename=sprintf("%s.png", filename))
  write.csv(D, file=sprintf("%s.csv", filename), row.names=FALSE, quote=FALSE)
  M$pred[, x := x * xscale]
  write.csv(M$pred, file=sprintf("%s_Pred.csv", filename), row.names=FALSE, quote=FALSE)
  show(file.info(c(sprintf("%s.png", filename), sprintf("%s.csv", filename)))[c("size", "mtime")])
  G
}

Dichotomize cumulative doses

* Cyclophosphamide
    * $\lt 2400$
    * $\ge 2400$
* Taxane
    * $\lt 500$
    * $\ge 500$

In [ ]:
D2 <- D1[,
         .(id = factor(id),
           authorYear,
           isCyclo,
           xCyc = cyclophosphamideCumulDose / 1e3,
           xCycHighDose = cyclophosphamideCumulDose >= 2400,
           xTax = taxaneCumulDose / 1e2,
           xTaxHighDose = taxaneCumulDose >= 500,
           isAnthra,
           isTaxane,
           isFluoro,
           nITT,
           medianFU,
           malType,
           malN,
           py,
           rate)]

### AML/MDS

* Fit the models
* Show model summaries
* Plot the predicted values

In [ ]:
mal <- "AML or MDS"
D3 <- D2[malType == mal]

In [ ]:
D3 <- D3[, `:=` (drug = "Cyclophosphamide", x = xCyc, xHighDose = xCycHighDose)]
M <- metareg(D3)
M
plotreg(M,
        D3,
        mal,
        "Cyclophosphamide cumulative dose",
        1e3 * c(0.5, 1, 2, 4, 8, 16),
        1e3)

In [ ]:
D3 <- D3[, `:=` (drug = "Taxane", x = xTax, xHighDose = xTaxHighDose)]
M <- metareg(D3)
M
plotreg(M,
        D3,
        mal,
        "Taxane cumulative dose",
        1e2 * c(0.5, 1, 2, 4, 8, 16),
        1e2)

#### Cyclophosphamide + Taxanes

The effect of cyclophosphamide could be due to taxanes in the regimen.

Fit the meta-regression model

$$\frac{y_i}{\text{py}_i} = \beta_0 + \beta_1 x_{\text{cyclophosphamide dose}, i} + \beta_2 I_{\text{taxane}, i} + \beta_3 x_{\text{cyclophosphamide dose}, i} I_{\text{taxane}, i} + \sigma_\text{study}$$

where $I_{\text{taxane}, i}$ is an 0/1 indicator for whether study $i$ has taxane in the regimen. The interpretation of $\beta_1$ is the effect of cyclophosphamide dose among the studies with no taxane in the regimen.

Next, the model is reparameterized as 

$$\frac{y_i}{\text{py}_i} = \beta_0 + \beta_1^\prime x_{\text{cyclophosphamide dose}, i} + \beta_2^\prime I_{\text{no taxane}, i} + \beta_3^\prime x_{\text{cyclophosphamide dose}, i} I_{\text{no taxane}, i} + \sigma_\text{study}$$

so the interpretation of $\beta_1^\prime$ is the effect of cyclophosphamide dose among the studies with taxane in the regimen.

In [ ]:
mal <- "AML or MDS"
D3 <- D2[malType == mal]
D3 <- D3[, drug := "Cyclophosphamide"]
D3 <- D3[, isNoTax := !isTaxane]
D3 <- D3[, isNoCyc := !isCyclo]
# D3 <- D3[isTaxane == FALSE, xTax := 0]
# D3 <- D3[isCyclo == FALSE, xCyc := 0]
D3 <- D3[isTaxane == FALSE, xTaxHighDose := FALSE]
D3 <- D3[isCyclo == FALSE, xCycHighDose := FALSE]
D3 <- D3[!(is.na(xCyc) | is.na(xCycHighDose) | is.na(rate) | is.na(nITT) | is.na(malType))]
xData <- unique(D3[, .(drug, xCyc, xCycHighDose, isTaxane, malType)])
D3 <- escalc("IRLN", xi=malN, ti=py, data=D3)
randomEffect <- list(~ 1 | id)
MLin <- rma.mv(yi ~ xCyc + isTaxane + xCyc * isTaxane,
               vi,
               random=randomEffect,
               data=D3)
MBin <- rma.mv(yi ~ xCycHighDose + isTaxane + xCycHighDose * isTaxane,
               vi,
               random=randomEffect,
               data=D3)
M <- list(rmaLin = MLin,
          rmaBin = MBin)
M

---

In [ ]:
pvalueLinWithoutTax <- MLin$pval[which(row.names(MLin$b) == "xCyc")]
pvalueLinWithTax <- MLin$pval[which(row.names(MLin$b) == "xCyc:isTaxaneTRUE")]
pvalueBinWithoutTax <- MBin$pval[which(row.names(MBin$b) == "xCycHighDoseTRUE")]
pvalueBinWithTax <- MBin$pval[which(row.names(MBin$b) == "xCycHighDoseTRUE:isTaxaneTRUE")]
D3 <- data.table(D3)
pred <- cbind(D3[, .(drug, malType, xCyc, xCycHighDose, isTaxane)],
              predLin = predict(MLin, transf = exp)[["pred"]] * scale, 
              confLowerLin = predict(MLin, transf = exp)[["cr.lb"]] * scale, 
              confUpperLin = predict(MLin, transf = exp)[["cr.ub"]] * scale, 
              predBin = predict(MBin, transf = exp)[["pred"]] * scale, 
              confLowerBin = predict(MBin, transf = exp)[["cr.lb"]] * scale, 
              confUpperBin = predict(MBin, transf = exp)[["cr.ub"]] * scale, 
              scale)
pred <- unique(pred)
setorder(pred, xCyc)
M <- list(rmaLin = MLin,
          rmaBin = MBin,
          pvalueLinWithTax = pvalToChar(pvalueLinWithTax),
          pvalueLinWithoutTax = pvalToChar(pvalueLinWithoutTax),
          pvalueBinWithTax = pvalToChar(pvalueBinWithTax),
          pvalueBinWithoutTax = pvalToChar(pvalueBinWithoutTax),
          pred = pred)
M

#### Taxanes + Cyclophosphamide

The effect of taxanes could be due to cyclophosphamide in the regimen.

Fit the meta-regression model

$$\frac{y_i}{\text{py}_i} = \beta_0 + \beta_1 I_{\text{no cyclophosphamide}, i} + \beta_2 x_{\text{taxane dose}, i} + \beta_3 x_{\text{taxane dose}, i} I_{\text{no cyclophosphamide}, i} + \sigma_\text{study}$$

In [ ]:
mal <- "AML or MDS"
D3 <- D2[malType == mal]
D3 <- D3[, drug := "Taxane"]
D3 <- D3[, isNoTax := !isTaxane]
D3 <- D3[, isNoCyc := !isCyclo]
# D3 <- D3[isTaxane == FALSE, xTax := 0]
# D3 <- D3[isCyclo == FALSE, xCyc := 0]
D3 <- D3[isTaxane == FALSE, xTaxHighDose := FALSE]
D3 <- D3[isCyclo == FALSE, xCycHighDose := FALSE]
D3 <- D3[!(is.na(xTax) | is.na(xTaxHighDose) | is.na(rate) | is.na(nITT) | is.na(malType))]
xData <- unique(D3[, .(drug, xTax, xTaxHighDose, isNoCyc, malType)])
D3 <- escalc("IRLN", xi=malN, ti=py, data=D3)
randomEffect <- list(~ 1 | id)
MLin <- rma.mv(yi ~ isNoCyc + xTax + xTax * isNoCyc,
               vi,
               random=randomEffect,
               data=D3)
MBin <- rma.mv(yi ~ isNoCyc + xTaxHighDose + xTaxHighDose * isNoCyc,
               vi,
               random=randomEffect,
               data=D3)
pvalueLinWithCyc <- MLin$pval[which(row.names(MLin$b) == "xTax")]
pvalueLinWithoutCyc <- MLin$pval[which(row.names(MLin$b) == "isNoCycTRUE:xTax")]
pvalueBinWithCyc <- MBin$pval[which(row.names(MBin$b) == "xTaxHighDoseTRUE")]
pvalueBinWithoutCyc <- MBin$pval[which(row.names(MBin$b) == "isNoCycTRUE:xTaxHighDoseTRUE")]
D3 <- data.table(D3)
pred <- cbind(D3[, .(drug, malType, isNoCyc, xTax, xTaxHighDose)],
              predLin = predict(MLin, transf = exp)[["pred"]] * scale, 
              confLowerLin = predict(MLin, transf = exp)[["cr.lb"]] * scale, 
              confUpperLin = predict(MLin, transf = exp)[["cr.ub"]] * scale, 
              predBin = predict(MBin, transf = exp)[["pred"]] * scale, 
              confLowerBin = predict(MBin, transf = exp)[["cr.lb"]] * scale, 
              confUpperBin = predict(MBin, transf = exp)[["cr.ub"]] * scale, 
              scale)
pred <- unique(pred)
setorder(pred, xTax)
M <- list(rmaLin = MLin,
          rmaBin = MBin,
          pvalueLinWithCyc = pvalToChar(pvalueLinWithCyc),
          pvalueLinWithoutCyc = pvalToChar(pvalueLinWithoutCyc),
          pvalueBinWithCyc = pvalToChar(pvalueBinWithCyc),
          pvalueBinWithoutCyc = pvalToChar(pvalueBinWithoutCyc),
          pred = pred)
M

In [ ]:
require(ggplot2)
require(RColorBrewer)
require(tools)
xscale <- 1e2
title <- mal
xbreaks <- 1e2 * c(0.5, 1, 2, 4, 8, 16)
xlab <- "Taxane cumulative dose"
G <- ggplot(D3, aes(x=xTax * xscale, y=rate + 1/2, size=nITT / min(nITT, na.rm=TRUE)))
G <- G + geom_point(alpha=1/3, position="jitter", aes(shape=isNoCyc))
G <- G + scale_x_log10(xlab, breaks=xbreaks)
G <- G + scale_y_log10(sprintf("Rate per %s person-years", format(scale, big.mark=",")))
G <- G + labs(title=title)
G <- G + theme_bw()
G <- G + theme(legend.position="none")
G
D3[, .(id, authorYear, isNoCyc, xTax, malType, malN, py, rate)][order(isNoCyc, xTax)]

In [ ]:
pvalues <- c(M$pvalueLinWithCyc, M$pvalueLinWithoutCyc, M$pvalueBinWithCyc, M$pvalueBinWithoutCyc)
x <- data.table(1,
                rep(c(TRUE, FALSE), each = 10),
                seq(min(M$rmaLin$X[, "xTax"]), max(M$rmaLin$X[, "xTax"]), length.out=10))
x <- as.matrix(x, ncol=3)
x <- cbind(x, V4 = x[, 2] * x[, 3])
rownames(x) <- 1:nrow(x)
colnames(x) <- colnames(M$rmaLin$X)
str(x)
str(M$rmaLin$X)
predict(M$rmaLin, matrix(x), transf = exp, addx=TRUE)
# predict(M$rmaLin, transf = exp, addx=TRUE)


In [ ]:
yhat1 <- unique(data.table(malType = mal,
                           x,
                           yhat = predict(M$rmaLin, x, transf = exp)[["pred"]] * scale))

In [ ]:
xHighDose <- as.logical(M$rmaBin$X[, "xHighDoseTRUE"])
yhat2 <- unique(data.table(malType = mal,
                           xHighDose,
                           yhat = predict(M$rmaBin, xHighDose, transf = exp)[["pred"]] * scale))
D <- D[!(is.na(x) | is.na(rate) | is.na(nITT) | is.na(malType))]
D <- D[, malType := droplevels(malType)]
steps <- merge(D[, .(min = min(x), max = max(x)), .(malType, xHighDose)],
               yhat2,
               by=c("malType", "xHighDose"))
steps <- melt(steps, id.vars=c("malType", "xHighDose", "yhat"), measure.vars=c("min", "max"), value.name="x")
annoLin <- data.frame(x=Inf, y=1.2, label=pvalues[c(1)], malType=levels(D[, malType]))
annoBin <- data.frame(x=Inf, y=1.4, label=pvalues[c(2)], malType=levels(D[, malType]))
pal <- brewer.pal(4, name="RdBu")
G <- ggplot(D, aes(x=x * xscale, y=rate + 1/2, size=nITT / min(nITT, na.rm=TRUE)))
G <- G + geom_point(alpha=1/3, position="jitter")
G <- G + geom_line(data=yhat1, aes(x=x * xscale, y=yhat), inherit.aes=FALSE, color=pal[4])
G <- G + geom_step(data=steps, aes(x=x * xscale, y=yhat), inherit.aes=FALSE, color=pal[1])
G <- G + geom_text(data=annoLin,
                   aes(x, y, label=label), inherit.aes=FALSE, hjust=1, color=pal[4])
G <- G + geom_text(data=annoBin,
                   aes(x, y, label=label), inherit.aes=FALSE, hjust=1, color=pal[1])
G <- G + scale_x_log10(xlab, breaks=xbreaks)
G <- G + scale_y_log10(sprintf("Rate per %s person-years", format(scale, big.mark=",")))
G <- G + labs(title=title)
G <- G + theme_bw()
G <- G + theme(legend.position="none")
filename <- gsub("\\s+",
                 "",
                 toTitleCase(paste(gsub("[[:punct:]]", "", title),
                                   gsub("cumulative dose", "", xlab),
                                   sep="_")))
ggsave(filename=sprintf("%s.png", filename))
write.csv(D, file=sprintf("%s.csv", filename), row.names=FALSE, quote=FALSE)
M$pred[, x := x * xscale]
write.csv(M$pred, file=sprintf("%s_Pred.csv", filename), row.names=FALSE, quote=FALSE)
show(file.info(c(sprintf("%s.png", filename), sprintf("%s.csv", filename)))[c("size", "mtime")])
G

### AML only

Run models for AML malignancies alone for compatibility with the SEER incidence statistics.

> From: Rivera, Donna (NIH/NCI) [F] [mailto:donna.rivera@nih.gov]   
> Sent: Monday, June 27, 2016 6:37 AM  
> To: Benjamin Chan; Barbara M. Galligan  
> Subject: RE: SEER Data for NCI secondary malignancies  
> 
> I just wanted to make sure you saw the **SEER statistics are only for AML and do not include MDS**. MDS is categorized separately and we could get that as well for reference if you would like.  There are other types such as AMoL and more rare versions ,if  you would like to look at those. Let me know if there are other types needed for your reference.

In [ ]:
mal <- "AML"
D3 <- D2[malType == mal]

In [ ]:
D3 <- D3[, `:=` (drug = "Cyclophosphamide", x = xCyc, xHighDose = xCycHighDose)]
M <- metareg(D3)
M
plotreg(M,
        D3,
        mal,
        "Cyclophosphamide cumulative dose",
        1e3 * c(0.5, 1, 2, 4, 8, 16),
        1e3)

In [ ]:
D3 <- D3[, `:=` (drug = "Taxane", x = xTax, xHighDose = xTaxHighDose)]
M <- metareg(D3)
M
plotreg(M,
        D3,
        mal,
        "Taxane cumulative dose",
        1e2 * c(0.5, 1, 2, 4, 8, 16),
        1e2)

### Non-Breast Solid

* Fit the models
* Show model summaries
* Plot the predicted values

In [ ]:
mal <- "Non-Breast Solid"
D3 <- D2[malType == mal]

In [ ]:
D3 <- D3[, `:=` (drug = "Cyclophosphamide", x = xCyc, xHighDose = xCycHighDose)]
M <- metareg(D3)
M
plotreg(M,
        D3,
        mal,
        "Cyclophosphamide cumulative dose",
        1e3 * c(0.5, 1, 2, 4, 8, 16),
        1e3)

In [ ]:
D3 <- D3[, `:=` (drug = "Taxane", x = xTax, xHighDose = xTaxHighDose)]
M <- metareg(D3)
M
plotreg(M,
        D3,
        mal,
        "Taxane cumulative dose",
        1e2 * c(0.5, 1, 2, 4, 8, 16),
        1e2)

## Study Characteristics and Outcomes

Populate Appendix Table 1.
Need columns for

* Study
* Country
* Median follow-up (months)
* Regimen name
* Cumulative dose
    * Anthracyclines
    * Cyclophosphamide
    * Taxanes
* N
* Person-years (the denominator for the incidence rate)
* AML/MDS count (the numerator for the incidence rate)
* AML/MDS incidence, per 10,000 person-years
* Non-breast solid count (the numerator for the incidence rate)
* Non-breast solid incidence, per 10,000 person-years

Output to an 

* Excel workbook, [appendixTableStudyCharacteristicsAndOutcomes.xlsx](appendixTableStudyCharacteristicsAndOutcomes.xlsx)
* CSV file, [appendixTableStudyCharacteristicsAndOutcomes.csv](appendixTableStudyCharacteristicsAndOutcomes.csv)

In [ ]:
library(xlsx)
library(IRdisplay)
library(xtable)
T <- D[,
       .(study = ifelse(!is.na(trial), paste(authorYear, trial), authorYear),
         country = NA,
         medianFU = round(medianFU),
         regimen,
         anthrCumulDose = round(anthracyclineCumulDose),
         cycloCumulaDose = round(cyclophosphamideCumulDose),
         taxaneCumulDose = round(taxaneCumulDose),
         nITT,
         py = nITT * (medianFU / 12),
         nAMLOrMDS = malAMLOrMDSTotal,
         incidenceAMLOrMDS = signif(malAMLOrMDSTotal / (nITT * (medianFU / 12)) * scale, digits=3),
         nNonBreastSolid = malNonBreastSolid,
         incidenceNonBreastSolid = signif(malNonBreastSolid / (nITT * (medianFU / 12)) * scale, digits=3))]
write.xlsx(T,
           "appendixTableStudyCharacteristicsAndOutcomes.xlsx",
           row.names=FALSE,
           showNA=FALSE)
write.csv(T,
          "appendixTableStudyCharacteristicsAndOutcomes.csv",
          quote=FALSE,
          row.names=FALSE)
file.info(grep("appendixTableStudyCharacteristicsAndOutcomes", list.files(), value=TRUE))[c("size", "mtime")]
T <- xtable(T, digits=c(rep(0, ncol(T) - 3), 0, 2, 0, 2))
# display_html(paste(capture.output(print(T, type = 'html')), collapse="", sep=""))

## Zip up assets

Zip up assets created by this notebook for transfer off of EC2.

In [ ]:
f <- "assets.zip"
assets <- grep("(csv)|(png)|(xlsx)|(md)$", list.files(), value=TRUE)
zip(f, assets)
file.info(f)[c("size", "mtime")]
unzip(f, list=TRUE)

## Session information

In [ ]:
sessionInfo()